# Initialize

In [86]:
import geopandas as gpd
import psycopg2
import folium
import folium.plugins
import fiona
import osmnx as ox
import networkx as nx
import operator
import json
import matplotlib.cm as cmx
import matplotlib.colors as colors
from decimal import Decimal



def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b


def style_function(feature, n_colors):
    try: 
        cid = feature['properties']['cid']
        return {
            'fillOpacity': 0.5,
            'weight': 0,
            'fillColor': '#red' if cid is None else "rgb{}".format(rgb(0, n_colors, cid))
        }
    except KeyError:
            return {
            'fillOpacity': 0.1,
            'weight': 0,
            'fillColor': 'red' 
        }

def init_style_function(n_colors):
    return lambda feature: style_function(feature, n_colors)

green = lambda x: {'fillOpacity': '0.5', 'fillColor': 'green', 'color':''}
red = lambda x: {'fillOpacity': '0.5', 'fillColor': 'red', 'color':''}
blue = lambda x: {'fillOpacity': '0.5', 'fillColor': 'blue', 'color':''}

# PATH of Manual AOI

In [9]:
manual_aois_geodataframe = gpd.read_file("../../data/manual_aoi_halle.geojson")

# Optimizer

In [87]:
manual_aois_geodataframe = manual_aois_geodataframe.to_crs({'init': 'epsg:4326'})
manual_aois_json = json.dumps(manual_aois_geodataframe.__geo_interface__)

# get bbox from manual aois
bbox = manual_aois_geodataframe.__geo_interface__['bbox']
# add crs 
bbox = bbox + tuple([4326])
manual_aois_area=0.0
result = []

manual_aois_geodataframe = gpd.read_file("../../data/manual_aoi_halle.geojson")
manual_aois_geodataframe = manual_aois_geodataframe.to_crs({'init': 'epsg:4326'})
manual_aois_json = json.dumps(manual_aois_geodataframe.__geo_interface__)
# get bbox from manual aois
bbox = manual_aois_geodataframe.__geo_interface__['bbox']
bbox = bbox + tuple([4326])

bbox_query = f"""
SELECT ST_Transform(ST_SetSRid(ST_MakeEnvelope{bbox}, 4326), 3857) AS geometry
"""

# clip the pois with the bbox
pois_query = f"""
WITH bbox as ({bbox_query})
SELECT pois.geometry 
FROM pois, bbox
WHERE ST_Within(pois.geometry, bbox.geometry);
"""

manual_aois_query  = f"""
WITH feature_collection AS ( 
    SELECT '{manual_aois_json}'::json AS features
) 
SELECT ST_Transform(ST_SetSRid(ST_GeomFromGeoJSON(feature_col->>'geometry'), 4326), 3857) AS geometry
FROM (
    SELECT json_array_elements(features->'features') AS feature_col
    FROM feature_collection
) AS subquery;
"""

with psycopg2.connect("") as conn:
    pois = gpd.read_postgis(pois_query, conn, geom_col='geometry')
    pois.crs = fiona.crs.from_epsg(3857)
    manual_aois = gpd.read_postgis(manual_aois_query, conn, geom_col='geometry')
    manual_aois.crs= fiona.crs.from_epsg(3857)

for i, manual_polygon in manual_aois.iterrows():
    manual_aois_area = manual_aois_area + manual_polygon['geometry'].area
    
for eps in range(5,100):
    for minpoints in range(1, 20):
        aois_query = f"""
        WITH bbox as ({bbox_query}), 
        hulls AS ( SELECT preclusters.hull as hull
        FROM bbox INNER JOIN preclusters ON ST_Intersects(preclusters.hull,bbox.geometry)
        ), smaller_clusters AS(
        SELECT pois.geometry, ST_ClusterDBSCAN(pois.geometry, eps := {eps}, minpoints := {minpoints}) over () AS cid
        FROM pois, hulls
        WHERE ST_Within(geometry, hulls.hull)
        )
        SELECT * FROM smaller_clusters WHERE cid IS NOT NULL
        UNION ALL
        SELECT ST_ConcaveHull(ST_Union(geometry),0.99), cid FROM smaller_clusters WHERE cid IS NOT NULL GROUP BY cid
        """
        
        without_water_query = f"""
        WITH aois AS ({aois_query})
        SELECT ST_Difference(aois.geometry, coalesce((
            SELECT ST_Union(way) AS geometry 
            FROM planet_osm_polygon
            WHERE (
                water IS NOT NULL OR waterway IS NOT NULL
                )
                AND (
                tunnel IS NULL OR tunnel = 'no'
                )
            AND ST_Intersects(way, aois.geometry)
        ), 'GEOMETRYCOLLECTION EMPTY'::geometry)) AS geometry
        FROM aois
    
        """
        
        sanitized_aois_query = f"""
        WITH aois AS ({without_water_query}),
        sanitized_aois AS(
            SELECT ST_Simplify((ST_Dump(ST_Union(geometry))).geom, 5) AS geometry FROM aois
        )
        SELECT * FROM sanitized_aois WHERE ST_IsValid(geometry) AND NOT ST_IsEmpty(geometry)
        """
        
        with psycopg2.connect("") as conn:
            sanitized_aois = gpd.read_postgis(sanitized_aois_query, conn, geom_col='geometry')
            sanitized_aois.crs = fiona.crs.from_epsg(3857)
            if (sanitized_aois.size == 0):
                # print(f"No AOIs is generated eps: {eps}, minpoints: {minpoints}")
                # It is assumed that for the rest of {minpoints} no AOIs is generated   
                break

        intersection_area = 0.0
        calculated_aois_area = 0.0
        
        for i, manual_polygon in manual_aois.iterrows():
            for aois_index, calculated_polygon in sanitized_aois.iterrows():
                if manual_polygon['geometry'].intersects(calculated_polygon['geometry']):
                    intersection_area = intersection_area + (manual_polygon['geometry'].intersection(calculated_polygon['geometry'])).area

        for k, s in sanitized_aois.iterrows():
            calculated_aois_area = calculated_aois_area + (s['geometry'].area)
        aois_count = aois_index + 1
        correctness = (intersection_area/ (calculated_aois_area + manual_aois_area - intersection_area)) * 100
        
        print(f"eps: {eps:{2}}, minpoints: {minpoints:{2}}, correctness: {Decimal(correctness):{6}.{5}}%, aois_count: {aois_count:{2}}")
        result.append({'eps': eps, 'minpoints': minpoints, 'correctness': correctness, 'aois_count': aois_count})
        

eps:  5, minpoints:  1, correctness: 26.604%, aois_count: 51
eps:  5, minpoints:  2, correctness: 28.458%, aois_count: 38
eps:  5, minpoints:  3, correctness: 27.827%, aois_count: 16
eps:  5, minpoints:  4, correctness: 26.393%, aois_count:  2
eps:  5, minpoints:  5, correctness: 26.393%, aois_count:  2
eps:  5, minpoints:  6, correctness: 26.346%, aois_count:  1
eps:  6, minpoints:  1, correctness: 26.962%, aois_count: 50
eps:  6, minpoints:  2, correctness: 28.840%, aois_count: 37
eps:  6, minpoints:  3, correctness: 28.273%, aois_count: 18
eps:  6, minpoints:  4, correctness: 26.672%, aois_count:  4
eps:  6, minpoints:  5, correctness: 26.393%, aois_count:  2
eps:  6, minpoints:  6, correctness: 26.346%, aois_count:  1
eps:  7, minpoints:  1, correctness: 27.223%, aois_count: 53
eps:  7, minpoints:  2, correctness: 29.113%, aois_count: 40
eps:  7, minpoints:  3, correctness: 28.551%, aois_count: 20
eps:  7, minpoints:  4, correctness: 26.723%, aois_count:  5
eps:  7, minpoints:  5, 

eps: 23, minpoints:  9, correctness: 1.0691%, aois_count:  1
eps: 23, minpoints: 10, correctness: 0.68538%, aois_count:  1
eps: 24, minpoints:  1, correctness: 44.265%, aois_count: 24
eps: 24, minpoints:  2, correctness: 46.648%, aois_count: 19
eps: 24, minpoints:  3, correctness: 47.130%, aois_count: 10
eps: 24, minpoints:  4, correctness: 43.732%, aois_count:  6
eps: 24, minpoints:  5, correctness: 39.403%, aois_count:  7
eps: 24, minpoints:  6, correctness: 37.762%, aois_count:  8
eps: 24, minpoints:  7, correctness: 33.734%, aois_count:  6
eps: 24, minpoints:  8, correctness: 2.8129%, aois_count:  3
eps: 24, minpoints:  9, correctness: 1.5697%, aois_count:  2
eps: 24, minpoints: 10, correctness: 1.0691%, aois_count:  1
eps: 25, minpoints:  1, correctness: 45.892%, aois_count: 22
eps: 25, minpoints:  2, correctness: 48.387%, aois_count: 18
eps: 25, minpoints:  3, correctness: 48.933%, aois_count:  9
eps: 25, minpoints:  4, correctness: 44.111%, aois_count:  6
eps: 25, minpoints:  5,

eps: 35, minpoints:  1, correctness: 45.793%, aois_count: 21
eps: 35, minpoints:  2, correctness: 48.231%, aois_count: 17
eps: 35, minpoints:  3, correctness: 48.707%, aois_count: 11
eps: 35, minpoints:  4, correctness: 50.302%, aois_count:  5
eps: 35, minpoints:  5, correctness: 50.360%, aois_count:  4
eps: 35, minpoints:  6, correctness: 46.353%, aois_count:  4
eps: 35, minpoints:  7, correctness: 43.388%, aois_count:  5
eps: 35, minpoints:  8, correctness: 38.873%, aois_count:  7
eps: 35, minpoints:  9, correctness: 9.9536%, aois_count:  4
eps: 35, minpoints: 10, correctness: 7.1051%, aois_count:  2
eps: 35, minpoints: 11, correctness: 6.7162%, aois_count:  2
eps: 35, minpoints: 12, correctness: 4.7432%, aois_count:  3
eps: 35, minpoints: 13, correctness: 2.5410%, aois_count:  1
eps: 35, minpoints: 14, correctness: 2.0020%, aois_count:  1
eps: 35, minpoints: 15, correctness: 1.9424%, aois_count:  1
eps: 35, minpoints: 16, correctness: 1.3665%, aois_count:  1
eps: 36, minpoints:  1, 

eps: 42, minpoints: 13, correctness: 8.1567%, aois_count:  2
eps: 42, minpoints: 14, correctness: 6.1423%, aois_count:  3
eps: 42, minpoints: 15, correctness: 4.9789%, aois_count:  1
eps: 42, minpoints: 16, correctness: 3.2297%, aois_count:  1
eps: 42, minpoints: 17, correctness: 2.9323%, aois_count:  1
eps: 42, minpoints: 18, correctness: 2.7156%, aois_count:  1
eps: 42, minpoints: 19, correctness: 2.7156%, aois_count:  1
eps: 43, minpoints:  1, correctness: 45.038%, aois_count: 19
eps: 43, minpoints:  2, correctness: 47.105%, aois_count: 16
eps: 43, minpoints:  3, correctness: 46.683%, aois_count: 12
eps: 43, minpoints:  4, correctness: 49.312%, aois_count:  6
eps: 43, minpoints:  5, correctness: 50.963%, aois_count:  5
eps: 43, minpoints:  6, correctness: 50.226%, aois_count:  3
eps: 43, minpoints:  7, correctness: 48.590%, aois_count:  3
eps: 43, minpoints:  8, correctness: 45.548%, aois_count:  4
eps: 43, minpoints:  9, correctness: 15.395%, aois_count:  3
eps: 43, minpoints: 10, 

eps: 49, minpoints: 17, correctness: 6.8497%, aois_count:  1
eps: 49, minpoints: 18, correctness: 5.9133%, aois_count:  1
eps: 49, minpoints: 19, correctness: 5.2153%, aois_count:  1
eps: 50, minpoints:  1, correctness: 45.038%, aois_count: 19
eps: 50, minpoints:  2, correctness: 47.105%, aois_count: 16
eps: 50, minpoints:  3, correctness: 46.683%, aois_count: 12
eps: 50, minpoints:  4, correctness: 48.922%, aois_count:  6
eps: 50, minpoints:  5, correctness: 50.008%, aois_count:  6
eps: 50, minpoints:  6, correctness: 50.855%, aois_count:  4
eps: 50, minpoints:  7, correctness: 50.226%, aois_count:  3
eps: 50, minpoints:  8, correctness: 50.220%, aois_count:  3
eps: 50, minpoints:  9, correctness: 20.699%, aois_count:  2
eps: 50, minpoints: 10, correctness: 18.961%, aois_count:  2
eps: 50, minpoints: 11, correctness: 15.929%, aois_count:  3
eps: 50, minpoints: 12, correctness: 15.080%, aois_count:  2
eps: 50, minpoints: 13, correctness: 13.789%, aois_count:  2
eps: 50, minpoints: 14, 

...

eps: 71, minpoints:  6, correctness: 52.309%, aois_count:  4
eps: 71, minpoints:  7, correctness: 52.309%, aois_count:  4
eps: 71, minpoints:  8, correctness: 52.598%, aois_count:  4
eps: 71, minpoints:  9, correctness: 55.515%, aois_count:  2
eps: 71, minpoints: 10, correctness: 53.159%, aois_count:  2
eps: 71, minpoints: 11, correctness: 53.159%, aois_count:  2
eps: 71, minpoints: 12, correctness: 52.859%, aois_count:  2
eps: 71, minpoints: 13, correctness: 24.598%, aois_count:  1
eps: 71, minpoints: 14, correctness: 22.628%, aois_count:  1
eps: 71, minpoints: 15, correctness: 22.628%, aois_count:  1
eps: 71, minpoints: 16, correctness: 22.021%, aois_count:  1
eps: 71, minpoints: 17, correctness: 19.992%, aois_count:  2
eps: 71, minpoints: 18, correctness: 19.992%, aois_count:  2
eps: 71, minpoints: 19, correctness: 19.518%, aois_count:  2
eps: 72, minpoints:  1, correctness: 47.393%, aois_count: 11
eps: 72, minpoints:  2, correctness: 48.338%, aois_count: 10
eps: 72, minpoints:  3, 

eps: 78, minpoints:  9, correctness: 51.276%, aois_count:  3
eps: 78, minpoints: 10, correctness: 54.284%, aois_count:  2
eps: 78, minpoints: 11, correctness: 52.879%, aois_count:  2
eps: 78, minpoints: 12, correctness: 52.879%, aois_count:  2
eps: 78, minpoints: 13, correctness: 25.703%, aois_count:  1
eps: 78, minpoints: 14, correctness: 24.370%, aois_count:  1
eps: 78, minpoints: 15, correctness: 24.045%, aois_count:  1
eps: 78, minpoints: 16, correctness: 24.045%, aois_count:  1
eps: 78, minpoints: 17, correctness: 23.483%, aois_count:  1
eps: 78, minpoints: 18, correctness: 22.619%, aois_count:  1
eps: 78, minpoints: 19, correctness: 19.643%, aois_count:  1
eps: 79, minpoints:  1, correctness: 45.068%, aois_count:  9
eps: 79, minpoints:  2, correctness: 45.068%, aois_count:  9
eps: 79, minpoints:  3, correctness: 44.784%, aois_count:  7
eps: 79, minpoints:  4, correctness: 43.115%, aois_count:  7
eps: 79, minpoints:  5, correctness: 48.630%, aois_count:  5
eps: 79, minpoints:  6, 

eps: 85, minpoints: 11, correctness: 52.862%, aois_count:  2
eps: 85, minpoints: 12, correctness: 52.862%, aois_count:  2
eps: 85, minpoints: 13, correctness: 29.123%, aois_count:  1
eps: 85, minpoints: 14, correctness: 26.249%, aois_count:  1
eps: 85, minpoints: 15, correctness: 25.941%, aois_count:  1
eps: 85, minpoints: 16, correctness: 25.853%, aois_count:  1
eps: 85, minpoints: 17, correctness: 25.853%, aois_count:  1
eps: 85, minpoints: 18, correctness: 25.139%, aois_count:  1
eps: 85, minpoints: 19, correctness: 24.590%, aois_count:  1
eps: 86, minpoints:  1, correctness: 45.068%, aois_count:  9
eps: 86, minpoints:  2, correctness: 45.068%, aois_count:  9
eps: 86, minpoints:  3, correctness: 44.784%, aois_count:  7
eps: 86, minpoints:  4, correctness: 43.115%, aois_count:  7
eps: 86, minpoints:  5, correctness: 47.302%, aois_count:  5
eps: 86, minpoints:  6, correctness: 49.344%, aois_count:  5
eps: 86, minpoints:  7, correctness: 48.410%, aois_count:  5
eps: 86, minpoints:  8, 

eps: 92, minpoints: 13, correctness: 28.781%, aois_count:  1
eps: 92, minpoints: 14, correctness: 25.476%, aois_count:  1
eps: 92, minpoints: 15, correctness: 25.476%, aois_count:  1
eps: 92, minpoints: 16, correctness: 25.476%, aois_count:  1
eps: 92, minpoints: 17, correctness: 25.445%, aois_count:  1
eps: 92, minpoints: 18, correctness: 25.573%, aois_count:  1
eps: 92, minpoints: 19, correctness: 25.573%, aois_count:  1
eps: 93, minpoints:  1, correctness: 43.388%, aois_count:  8
eps: 93, minpoints:  2, correctness: 43.388%, aois_count:  8
eps: 93, minpoints:  3, correctness: 43.107%, aois_count:  6
eps: 93, minpoints:  4, correctness: 42.314%, aois_count:  6
eps: 93, minpoints:  5, correctness: 46.242%, aois_count:  5
eps: 93, minpoints:  6, correctness: 48.631%, aois_count:  5
eps: 93, minpoints:  7, correctness: 49.228%, aois_count:  5
eps: 93, minpoints:  8, correctness: 49.351%, aois_count:  5
eps: 93, minpoints:  9, correctness: 48.410%, aois_count:  5
eps: 93, minpoints: 10, 

eps: 99, minpoints: 15, correctness: 28.759%, aois_count:  1
eps: 99, minpoints: 16, correctness: 26.013%, aois_count:  1
eps: 99, minpoints: 17, correctness: 26.013%, aois_count:  1
eps: 99, minpoints: 18, correctness: 26.013%, aois_count:  1
eps: 99, minpoints: 19, correctness: 26.094%, aois_count:  1


# Optimized Parameter

In [39]:
result.sort(key=lambda x: x['correctness'], reverse=True)
result[0]

{'eps': 64, 'minpoint': 8, 'correctness': 55.8791912065431, 'aois_count': 3}

In [38]:
result

[{'eps': 64, 'minpoint': 8, 'correctness': 55.8791912065431, 'aois_count': 3},
 {'eps': 65, 'minpoint': 8, 'correctness': 55.8791912065431, 'aois_count': 3},
 {'eps': 66, 'minpoint': 8, 'correctness': 55.74112332746044, 'aois_count': 3},
 {'eps': 67, 'minpoint': 8, 'correctness': 55.74112332746044, 'aois_count': 3},
 {'eps': 69,
  'minpoint': 9,
  'correctness': 55.514979126484135,
  'aois_count': 2},
 {'eps': 70,
  'minpoint': 9,
  'correctness': 55.514979126484135,
  'aois_count': 2},
 {'eps': 71,
  'minpoint': 9,
  'correctness': 55.514979126484135,
  'aois_count': 2},
 {'eps': 72,
  'minpoint': 9,
  'correctness': 55.514979126484135,
  'aois_count': 2},
 {'eps': 73,
  'minpoint': 9,
  'correctness': 55.514979126484135,
  'aois_count': 2},
 {'eps': 74,
  'minpoint': 9,
  'correctness': 55.514979126484135,
  'aois_count': 2},
 {'eps': 63, 'minpoint': 7, 'correctness': 54.64706144657227, 'aois_count': 4},
 {'eps': 78,
  'minpoint': 10,
  'correctness': 54.28373211656064,
  'aois_count

# Graphical Representations of Optimized Results

In [41]:
eps = result[0]['eps']
minpoints = result[0]['minpoints']

# clip the pois with the bbox
pois_query = f"""
WITH bbox as ({bbox_query})
SELECT pois.geometry 
FROM pois, bbox
WHERE ST_Within(pois.geometry, bbox.geometry);
"""

aois_query = f"""
WITH bbox as ({bbox_query}), 
hulls AS ( SELECT preclusters.hull as hull
FROM bbox INNER JOIN preclusters ON ST_Intersects(preclusters.hull,bbox.geometry)
), smaller_clusters AS(
SELECT pois.geometry, ST_ClusterDBSCAN(pois.geometry, eps := {eps}, minpoints := {minpoints}) over () AS cid
FROM pois, hulls
WHERE ST_Within(geometry, hulls.hull)
)
SELECT * FROM smaller_clusters WHERE cid IS NOT NULL
UNION ALL
SELECT ST_ConcaveHull(ST_Union(geometry),0.99), cid FROM smaller_clusters WHERE cid IS NOT NULL GROUP BY cid
"""

without_water_query = f"""
WITH aois AS ({aois_query})
SELECT ST_Difference(aois.geometry, coalesce((
    SELECT ST_Union(way) AS geometry 
    FROM planet_osm_polygon
    WHERE (
        water IS NOT NULL OR waterway IS NOT NULL
        )
        AND (
        tunnel IS NULL OR tunnel = 'no'
        )
    AND ST_Intersects(way, aois.geometry)
), 'GEOMETRYCOLLECTION EMPTY'::geometry)) AS geometry
FROM aois

"""

sanitized_aois_query = f"""
WITH aois AS ({without_water_query}),
sanitized_aois AS(
    SELECT ST_Simplify((ST_Dump(ST_Union(geometry))).geom, 5) AS geometry FROM aois
)
SELECT * FROM sanitized_aois WHERE ST_IsValid(geometry) AND NOT ST_IsEmpty(geometry)
"""
        
        
with psycopg2.connect("") as conn:
    sanitized_aois = gpd.read_postgis(sanitized_aois_query, conn, geom_col='geometry')
    sanitized_aois.crs = fiona.crs.from_epsg(3857)
    pois = gpd.read_postgis(pois_query, conn, geom_col='geometry')
    pois.crs = fiona.crs.from_epsg(3857)
    sanitized_aois = gpd.read_postgis(sanitized_aois_query, conn, geom_col='geometry')
    sanitized_aois.crs = fiona.crs.from_epsg(3857)

default_view = list(manual_aois_geodataframe['geometry'].centroid.to_crs(epsg=4326).loc[0].coords)[0][::-1]

m = folium.Map(location=default_view, zoom_start=15, tiles="cartodbpositron")

folium.plugins.Fullscreen().add_to(m)

folium.GeoJson(manual_aois, style_function=red).add_to(m)
folium.GeoJson(sanitized_aois, style_function=blue).add_to(m)
folium.GeoJson(pois, style_function=green).add_to(m)

m

